In [1]:
import pandas as pd
import sys
import os
sys.path.append("../")
sys.path.append("../..")
from KETIPreDataIngestion.KETI_setting import influx_setting_KETI as ins

# Data Ingestion Initialization

## DB Client 

In [2]:
version = 1 # or 1

if version == 1: 
    from KETIPreDataIngestion.data_influx import influx_Client as influx_Client
    dataServer = ins.CLUSTDataServer
elif version == 2:
    from KETIPreDataIngestion.data_influx import influx_Client_v2 as influx_Client
    dataServer = ins.CLUSTDataServer2

DBClient = influx_Client.influxClient(dataServer)

In [3]:
db_list = DBClient.get_DBList()

## DB MS Frequency

In [4]:
db_name ='air_indoor_중학교'
ms_name = DBClient.measurement_list(db_name)[0]

In [5]:
freq = DBClient.get_freq(db_name, ms_name)
print(freq)

{'freq': '0 days 00:01:00'}


## DB and Measurement name setting

In [50]:
db_name  = 'bio_covid_infected_world'

In [51]:
ms_list = DBClient.measurement_list(db_name)
print(ms_list)

['argentina', 'australia', 'england', 'france', 'germany', 'japan', 'korea', 'south_africa', 'spain', 'usa']


In [52]:
ms_list_only = DBClient.measurement_list_only_start_end(db_name)
ms_list_only

['argentina', '...(+8)', 'usa']

In [53]:
ms_name = 'england'

# Data Ingestion Methods

## pure data ingestion

### Ingest Full Data

In [9]:
print(db_name, ms_name)

bio_covid_infected_world england


In [10]:
feature_list = DBClient.get_fieldList(db_name, ms_name)
print(feature_list)

['confirmed', 'deaths']


In [35]:
full_data = DBClient.get_data(db_name, ms_name)
full_data

,confirmed,deaths
time,,
2020-01-22 00:00:00+00:00,0,0
2020-01-31 00:00:00+00:00,2,0
2020-02-03 00:00:00+00:00,8,0
2020-02-05 00:00:00+00:00,9,0
2020-02-08 00:00:00+00:00,13,0
...,...,...
2021-11-03 00:00:00+00:00,9215683,141607
2021-11-04 00:00:00+00:00,9252646,141826
2021-11-05 00:00:00+00:00,9286618,142019


In [31]:
number = 100
data_front_number = DBClient.get_datafront_by_num(number, db_name, ms_name)
data_front_number

,confirmed,deaths
time,,
2020-01-22 00:00:00+00:00,0,0
2020-01-31 00:00:00+00:00,2,0
2020-02-03 00:00:00+00:00,8,0
2020-02-05 00:00:00+00:00,9,0
2020-02-08 00:00:00+00:00,13,0
...,...,...
2020-04-26 00:00:00+00:00,158926,24053
2020-04-27 00:00:00+00:00,163630,24376
2020-04-28 00:00:00+00:00,168357,25347


In [28]:
number = 100
data_end_number = DBClient.get_dataend_by_num(number, db_name, ms_name)
data_end_number

,confirmed,deaths
time,,
2021-07-31 00:00:00+00:00,5883421,129949
2021-08-01 00:00:00+00:00,5907594,130014
2021-08-02 00:00:00+00:00,5929881,130039
2021-08-03 00:00:00+00:00,5951736,130179
2021-08-04 00:00:00+00:00,5980887,130300
...,...,...
2021-11-03 00:00:00+00:00,9215683,141607
2021-11-04 00:00:00+00:00,9252646,141826
2021-11-05 00:00:00+00:00,9286618,142019


In [34]:
first = DBClient.get_first_time(db_name, ms_name)
last = DBClient.get_last_time(db_name, ms_name)
print(first)
print(last)

2020-01-22T00:00:00Z
2021-11-07T00:00:00Z


In [36]:
query_end_time = last
query_start_time = first
bind_params = {'end_time':query_end_time, 'start_time': query_start_time}
time_data = DBClient.get_data_by_time(bind_params, db_name, ms_name)
print(time_data.head())
print(time_data.tail())

                           confirmed  deaths
time                                        
2020-01-22 00:00:00+00:00          0       0
2020-01-31 00:00:00+00:00          2       0
2020-02-03 00:00:00+00:00          8       0
2020-02-05 00:00:00+00:00          9       0
2020-02-08 00:00:00+00:00         13       0
                           confirmed  deaths
time                                        
2021-11-02 00:00:00+00:00    9174153  141390
2021-11-03 00:00:00+00:00    9215683  141607
2021-11-04 00:00:00+00:00    9252646  141826
2021-11-05 00:00:00+00:00    9286618  142019
2021-11-06 00:00:00+00:00    9317072  142174


In [55]:
days = 7 
bind_params = {'end_time': query_end_time,"days":str(days)+"d"}
day_data = DBClient.get_data_by_days(bind_params, db_name, ms_name)
print(day_data.head())
print(day_data.tail())

                           confirmed  deaths
time                                        
2021-10-31 00:00:00+00:00    9100442  141055
2021-11-01 00:00:00+00:00    9140441  141098
2021-11-02 00:00:00+00:00    9174153  141390
2021-11-03 00:00:00+00:00    9215683  141607
2021-11-04 00:00:00+00:00    9252646  141826
                           confirmed  deaths
time                                        
2021-11-03 00:00:00+00:00    9215683  141607
2021-11-04 00:00:00+00:00    9252646  141826
2021-11-05 00:00:00+00:00    9286618  142019
2021-11-06 00:00:00+00:00    9317072  142174
2021-11-07 00:00:00+00:00    9346961  142236


## Use 2.0.9 version

In [ ]:
query_end_time = last
query_start_time = first
number = 100
bind_params = {'end_time':query_end_time, 'start_time': query_start_time}
data_limit_time = DBClient.get_data_limit_by_time(bind_params, number, db_name, ms_name)
data_limit_time

In [54]:
data_count = DBClient.get_data_count(db_name, ms_name)
data_count

AttributeError: 'influxClient' object has no attribute 'get_data_count'

## Tag methods

In [39]:
db_name  = 'finance_korean_stock'

In [40]:
ms_list = DBClient.measurement_list(db_name)
print(ms_list)

['stock']


In [41]:
ms_name = 'stock'

In [42]:
tag_list = DBClient.get_tagList(db_name, ms_name)
print(tag_list)

['company', 'country', 'exchange', 'industry', 'ticker']


In [47]:
tag_key = 'company'
tag_distinct_value = DBClient.get_TagValue(db_name, ms_name, tag_key)
tag_distinct_value

['AJ네트웍스',
 'ASML 홀딩 NV ADR',
 'AT&T 5.625% 글로벌 노트',
 'AT&T Inc',
 'Abb Ltd ADR',
 'Afya Ltd Class A',
 'Alliance Resource Partners LP Unit',
 'BGF리테일',
 'BHP 그룹',
 'BHP 그룹 PLC ADR',
 'BP PLC ADR',
 'Brookfield Business Partners LP Unit',
 'CAE Inc',
 'CBRE 그룹 Class A',
 'CJ',
 'CJ CGV',
 'CJ대한통운',
 'CJ제일제당',
 'CMS 에너지',
 'CSX Corp',
 'CVS 헬스',
 'D.R.호턴',
 'DB',
 'DB손해보험',
 'DB하이텍',
 'DL이앤씨',
 'ESR켄달스퀘어리츠',
 'Enterprise Products Partners LP Unit',
 'F&F',
 'GKL',
 'GS',
 'GSX 테크에듀 ADR Class A',
 'GS건설',
 'GS글로벌',
 'GS리테일',
 'HCA 헬스케어',
 'HMM',
 'IPG 포터닉스',
 'JD닷컴 ADR',
 'KB금융',
 'KCC',
 'KG동부제철',
 'KG케미칼',
 'KODEX 200',
 'KODEX 200선물인버스2X',
 'KODEX 단기채권',
 'KODEX 삼성그룹',
 'KSS해운',
 'KT',
 'KT&G',
 'KTcs',
 'LG',
 'LG디스플레이',
 'LG상사',
 'LG생활건강',
 'LG생활건강우',
 'LG유플러스',
 'LG이노텍',
 'LG전자',
 'LG전자우',
 'LG헬로비전',
 'LG화학',
 'LIG넥스원',
 'MPLX LP Unit',
 'MYT 네덜란드 패어런트 ADR',
 'NAVER',
 'NHN',
 'NICE',
 'NIO ADR Class A',
 'Nuveen Amt-Free Quality Municipal Income Fund',
 'PC 커넥션',
 'PCN 파이낸셜 서비스',


In [48]:
tag_value='컴캐스트'
tag_value_data = DBClient.get_TagGroupData(db_name, ms_name, tag_key, tag_value)
tag_value_data

,adj_close_value,close_value,company,country,exchange,high_value,industry,low_value,open_value,ticker,volume_value
time,,,,,,,,,,,
2021-12-01 00:00:00+00:00,49.080002,49.080002,컴캐스트,US,NASDAQ,51.400002,미디어 및 출판,49.049999,51.134899,CMCSA,20791588.0
2021-12-16 00:00:00+00:00,48.840000,48.840000,컴캐스트,US,NASDAQ,49.520000,미디어 및 출판,48.419998,48.549999,CMCSA,26752590.0
2021-12-17 00:00:00+00:00,48.750000,48.750000,컴캐스트,US,NASDAQ,49.250000,미디어 및 출판,48.040001,48.209999,CMCSA,41501885.0
2021-12-20 00:00:00+00:00,48.400002,48.400002,컴캐스트,US,NASDAQ,48.674999,미디어 및 출판,47.980000,48.250000,CMCSA,18991157.0
2021-12-21 00:00:00+00:00,48.860001,48.860001,컴캐스트,US,NASDAQ,49.314999,미디어 및 출판,48.400002,48.630001,CMCSA,19714355.0
2021-12-22 00:00:00+00:00,49.029999,49.029999,컴캐스트,US,NASDAQ,49.160000,미디어 및 출판,48.424999,48.779999,CMCSA,24115443.0
2021-12-23 00:00:00+00:00,50.029999,50.029999,컴캐스트,US,NASDAQ,50.215000,미디어 및 출판,49.320000,49.369999,CMCSA,18420036.0
2021-12-27 00:00:00+00:00,50.470001,50.470001,컴캐스트,US,NASDAQ,50.590000,미디어 및 출판,49.660000,50.020000,CMCSA,16422831.0
2021-12-28 00:00:00+00:00,50.509998,50.509998,컴캐스트,US,NASDAQ,51.020000,미디어 및 출판,50.310001,50.470001,CMCSA,9285440.0
